In [ ]:
"""
TODO: 
CAGE & XPRESSO: MERGE ON CHROMOSOME AND TSS (TSS_xpresso from Xpresso csv)
"""


In [2]:
import sys
sys.path.append("/home/coder/diffTSS")

In [3]:
import os
import re
import glob
import kipoiseq
import numpy as np
import pandas as pd
import polars as pl
from tqdm import tqdm
from scipy import stats
import matplotlib.pyplot as plt
from scripts.utils import FastaStringExtractor, one_hot_encode

In [4]:
strand_pairs = {
    # GM12878
    "ENCFF074SXQ": "ENCFF164VLA", 
    "ENCFF546NVF": "ENCFF182LTN", 
    "ENCFF078ATR": "ENCFF037DUE", 
    "ENCFF892WMR": "ENCFF985TNZ", 

    # K562
    "ENCFF829PNJ": "ENCFF336COA", 
    "ENCFF964BAP": "ENCFF006DQI", 
    "ENCFF777EAJ": "ENCFF040DXX", 
    "ENCFF528VFJ": "ENCFF097ASF", 
}

In [92]:
data = os.listdir("/home/coder/data/RNASeq_bw")
tab_files = sorted([file for file in data if file.endswith(".tab") and "gene_added" not in file], key=lambda x: (x[x.find(".") + 1], x[0]))
tab_files = tab_files[0:16] + tab_files[24:32]
coverage_files = sorted([file for file in data if file.endswith(".txt")], key=lambda x: (x[x.find(".") + 1], x[0]))

combined_data = [file for file in glob.glob1("/home/coder/data/RNASeq_bw", "*stranded*") if "unstranded" not in file]
combined_tab_files = [file for file in combined_data if file.endswith(".tab")]
combined_coverage_files = [file for file in combined_data if file.endswith(".txt")]

combined_tab_files_names = [file[ :file.find("_")] for file in combined_tab_files]
combined_tab_files_name_sorted = sorted(combined_tab_files, key=lambda x: x[0])
combined_tab_files_cell_line_sorted = sorted(combined_tab_files, key=lambda x: x[0])

stranded_tab_files = [file for file in tab_files if "unstranded" not in file]
cell_line_sorted_files = sorted(tab_files, key=lambda x: x[0])

unstranded_tab_files = sorted(glob.glob1("/home/coder/data/RNASeq_bw", "*unstranded*.tab"), key=lambda x: x[0])
stranded_unstranded_tab = sorted(combined_tab_files_cell_line_sorted + unstranded_tab_files, key=lambda x: x[0])

all_types_tab = sorted(tab_files[:-8] + stranded_unstranded_tab, key=lambda x: (x[0], x[x.find(".") + 1: x.find(".") + 2]))

gene_added_files = glob.glob("gene_added*", root_dir="/home/coder/data/RNASeq_bw")

## TAB FILES

In [ ]:
# Plotting the plus and minus genes for each .tab file.
for file in tab_files:
    file_name = file[ :file.find("_")]
    df = pd.DataFrame(pl.read_csv(f"/home/coder/data/RNASeq_bw/{file}", separator="\t", skip_rows=3))

    # Distribution of Averaged Values
    df.mean().plot(kind="line")
    plt.xticks(range(0, 125, 5))
    plt.xlabel("Bins")
    plt.ylabel("Averaged Value")
    plt.title(f"Distribution of Averaged Bins for {file_name}")
    plt.savefig(f"/home/coder/REU/plots/dist_sample/{file_name}_averaged_values.png")
    plt.clf()

<Figure size 640x480 with 0 Axes>

In [102]:
# Plotting the combined plus and minus strands for each .tab file.
for file in combined_tab_files:
    file_name = file[ :file.find("_")]
    df = pd.DataFrame(pl.read_csv(f"/home/coder/data/RNASeq_bw/{file}", separator="\t", skip_rows=3)).iloc[ :, :125]

    # Distribution of Summed Values
    df.sum().plot(kind="line")
    plt.xticks(range(0, 125, 5))
    plt.xlabel("Bins")
    plt.ylabel("Summed Value")
    plt.title(f"Distribution of Summed Bins for {file_name}", fontsize=9)
    plt.savefig(f"/home/coder/REU/plots/{file_name}.paired_summed_values.png")
    plt.clf()

    # Distribution of Averaged Values
    df.mean().plot(kind="line")
    plt.xticks(range(0, 125, 5))
    plt.xlabel("Bins")
    plt.ylabel("Averaged Value")
    plt.title(f"Distribution of Averaged Bins for {file_name}", fontsize=9)
    plt.savefig(f"/home/coder/REU/plots/dist_combined/{file_name}.paired_averaged_values.png")
    plt.clf()

<Figure size 640x480 with 0 Axes>

In [99]:
# Plotting the average of each cell line and each strand type for .tab files.
for i in range(4):
    fig = plt.figure()
    same_strands = stranded_tab_files[i * 4: i * 4 + 4]
    strand_type = re.search(r"m[a-z]+|p[a-z]+", same_strands[0]).group(0)
    cell_line = same_strands[0][ :same_strands[0].find(".")]
    for file in same_strands:
        file_name = file[file.find("E"): file.find("_")]
        df = pd.DataFrame(pl.read_csv(f"/home/coder/data/RNASeq_bw/{file}", separator="\t", skip_rows=3))
        plt.plot(df.columns, df.mean(), label=file_name)

    plt.xticks(range(0, 125, 5))
    plt.xlabel("Bins")
    plt.ylabel("Averaged Value")
    plt.legend()
    plt.title(f"Distribution of Averaged Bins for {cell_line}.{strand_type}")
    plt.savefig(f"/home/coder/REU/plots/dist_sample_strand/{cell_line}.{strand_type}.averaged_values.png")
    plt.clf()

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [98]:
# Plotting the averaged values for each cell line.
for i in range(2):
    same_cell_lines = cell_line_sorted_files[i * 12: i * 12 + 12]
    cell_line = same_cell_lines[0][ :same_cell_lines[0].find(".")]
    for file in same_cell_lines:
        file_name = file[ :file.find("_")]
        df = pd.DataFrame(pl.read_csv(f"/home/coder/data/RNASeq_bw/{file}", separator="\t", skip_rows=3))
        plt.plot(df.columns, df.mean(), label=file_name)
        
    plt.xticks(range(0, 125, 5))
    plt.xlabel("Bins")
    plt.ylabel("Averaged Value")
    plt.legend(fontsize=7)
    plt.title(f"Distribution of Averaged Bins for {cell_line}")
    plt.savefig(f"/home/coder/REU/plots/dist_sample_cell_line/{cell_line}.averaged_values.png")
    plt.clf()

<Figure size 640x480 with 0 Axes>

In [97]:
cell_line_sorted_files

['GM12878.minus.ENCFF074SXQ_values_TSS.tab',
 'GM12878.minus.ENCFF078ATR_values_TSS.tab',
 'GM12878.minus.ENCFF546NVF_values_TSS.tab',
 'GM12878.minus.ENCFF892WMR_values_TSS.tab',
 'GM12878.plus.ENCFF037DUE_values_TSS.tab',
 'GM12878.plus.ENCFF164VLA_values_TSS.tab',
 'GM12878.plus.ENCFF182LTN_values_TSS.tab',
 'GM12878.plus.ENCFF985TNZ_values_TSS.tab',
 'GM12878.unstranded.ENCFF104OTO_values_TSS.tab',
 'GM12878.unstranded.ENCFF395OHI_values_TSS.tab',
 'GM12878.unstranded.ENCFF155XJQ_values_TSS.tab',
 'GM12878.unstranded.ENCFF143BSQ_values_TSS.tab',
 'K562.minus.ENCFF528VFJ_values_TSS.tab',
 'K562.minus.ENCFF829PNJ_values_TSS.tab',
 'K562.minus.ENCFF964BAP_values_TSS.tab',
 'K562.minus.ENCFF777EAJ_values_TSS.tab',
 'K562.plus.ENCFF006DQI_values_TSS.tab',
 'K562.plus.ENCFF097ASF_values_TSS.tab',
 'K562.plus.ENCFF336COA_values_TSS.tab',
 'K562.plus.ENCFF040DXX_values_TSS.tab',
 'K562.unstranded.ENCFF448XCV_values_TSS.tab',
 'K562.unstranded.ENCFF451LAH_values_TSS.tab',
 'K562.unstranded.

In [95]:
# Plotting file types (plus, minus, stranded, unstranded) against each other.
for i in range(2):
    same_cell_line = all_types_tab[i * 16: i * 16 + 16]
    cell_line = same_cell_line[0][ :same_cell_line[0].find(".")]
    for i in range(4):
        same_type_df = pd.DataFrame()
        same_type = same_cell_line[i * 4: i * 4 + 4]
        file_type = same_type[0][same_type[0].find(".") + 1: same_type[0].find("E") - 1].upper()

        for file in same_type:
            if file_type == "STRANDED":
                file_df = pd.read_csv(f"/home/coder/data/RNASeq_bw/{file}", sep="\t").iloc[:, :125]
            else:
                file_df = pd.read_csv(f"/home/coder/data/RNASeq_bw/{file}", sep="\t", skiprows=3, header=None)
            same_type_df = pd.concat([same_type_df, file_df])
        plt.plot(same_type_df.columns, same_type_df.mean(), label=file_type)
    plt.xticks(range(0, 125, 10))
    plt.xlabel("Bins")
    plt.ylabel("Average RNA Seq Value")
    plt.title(f"Minus vs Plus vs Stranded vs Unstranded for {cell_line}")
    plt.legend()
    plt.savefig(f"/home/coder/REU/plots/dist_type/{cell_line}.types.png")
    plt.clf()

<Figure size 640x480 with 0 Axes>

In [91]:
# Plotting the averaged values for each combined cell line.
for i in range(2):
    same_cell_lines = stranded_unstranded_tab[i * 8: i * 8 + 8]
    cell_line = same_cell_lines[0][ :same_cell_lines[0].find(".")]
    for file in same_cell_lines:
        file_name = file[ :file.find("_")]
        df = pd.DataFrame(pl.read_csv(f"/home/coder/data/RNASeq_bw/{file}", separator="\t", skip_rows=3))
        if "unstranded" in file:
            plt.plot(df.columns, df.mean(), linestyle="dotted", label=file_name)
        else:
            plt.plot(df.columns, df.mean(), linestyle="solid", label=file_name)
        
    plt.xticks(range(0, 125, 5))
    plt.xlabel("Bins")
    plt.ylabel("Averaged Value")
    plt.legend(fontsize=6)
    plt.title(f"Distribution of Averaged Bins for {cell_line}", fontsize=9)
    plt.savefig(f"/home/coder/REU/plots/dist_stranded_cell_line/{cell_line}.paired.averaged_values.png")
    plt.clf()

<Figure size 640x480 with 0 Axes>

In [67]:
# Plotting the average of each stranded pair.
for i in range(len(stranded_tab_files) // 2):
    minus_name = re.search(r"E[a-zA-Z0-9]+", stranded_tab_files[i]).group(0)
    cell_line = stranded_tab_files[i][ :stranded_tab_files[i].find(".")]
    for j in range(len(stranded_tab_files) // 2, len(stranded_tab_files)):
        plus_name = re.search(r"E[a-zA-Z0-9]+", stranded_tab_files[j]).group(0)

        if strand_pairs[minus_name] == plus_name:
            minus_strand = pd.DataFrame(pl.read_csv(f"/home/coder/data/RNASeq_bw/{stranded_tab_files[i]}", separator="\t", skip_rows=3))
            plus_strand = pd.DataFrame(pl.read_csv(f"/home/coder/data/RNASeq_bw/{stranded_tab_files[j]}", separator="\t", skip_rows=3))

            plt.plot(minus_strand.columns, minus_strand.mean(), label=minus_name, color="blue")
            plt.plot(plus_strand.columns, plus_strand.mean(), label=plus_name, color="orange")

            plt.xticks(range(0, 125, 5))
            plt.xlabel("Bins")
            plt.ylabel("Averaged Value")
            plt.legend()
            plt.title(f"Distribution of Averaged Bins for {cell_line} Strand Pair: {plus_name} and {minus_name}", fontsize=9)
            plt.savefig(f"/home/coder/REU/plots/plus_vs_minus_strand/{cell_line}.{plus_name}.{minus_name}.averaged_values.png")
            plt.clf()

<Figure size 640x480 with 0 Axes>

In [66]:
for file in gene_added_files:
    file_name = file[file.find("d") + 5: file.find("v") - 1]
    gene_added_df = pd.read_csv(f"/home/coder/data/RNASeq_bw/{file}", sep="\t", skiprows=3, header=None)
    tss_bed = pd.read_csv("/home/coder/data/TSS.bed", sep="\t", skiprows=1, header=None)

    gene_added_df["+/-"] = tss_bed[5]
    gene_added_plus = gene_added_df[gene_added_df["+/-"] == "+"]
    gene_added_minus = gene_added_df[gene_added_df["+/-"] == "-"]

    plt.plot(range(0, 125), gene_added_minus.iloc[:, 1:126].mean(), label="Minus", color="blue")
    plt.plot(range(0, 125), gene_added_plus.iloc[:, 1:126].mean(), label="Plus", color="orange")
    plt.xlabel("Bins")
    plt.ylabel("Average RNA Seq Value")
    plt.title(f"Plus vs Minus Strands for {file_name}")
    plt.legend()
    plt.savefig(f"/home/coder/REU/plots/plus_vs_minus_binned/{file_name}.plus_minus_average_binned.png")
    plt.clf()

<Figure size 640x480 with 0 Axes>

In [61]:
# Plotting the sum of the 3 bins around the TSS vs each gene.

GM12878_files = []
K562_files = []

GM12878_gene_list = pd.read_csv("/home/coder/data/ABC-multiTSS_nominated/GM12878/Neighborhoods/GeneList.txt", sep="\t")
K562_gene_list = pd.read_csv("/home/coder/data/ABC-multiTSS_nominated/K562/Neighborhoods/GeneList.txt", sep="\t")
rna_cage = pd.read_csv("/home/coder/data/RNA_CAGE.txt", sep="\t")

with pd.option_context("mode.chained_assignment", None):
    for file in tqdm(combined_tab_files, desc="File"):
        file_name = file[ :file.find("_")]
        cell_line = file[ :file.find(".")]
        gene_list = None

        if cell_line == "K562":
            gene_list = K562_gene_list
        elif cell_line == "GM12878":
            gene_list = GM12878_gene_list

        df = pd.read_csv(f"/home/coder/data/RNASeq_bw/{file}", sep="\t", skiprows=3, header=None)

        # Summing across the 3 bins surround the TSS.
        # df_tss = df.iloc[:, 62:65]
        # df_tss.loc[:, "ENSID"] = gene_list.loc[:, "Ensembl_ID"]
        # df_tss.loc[:, "Sum"] = df_tss.iloc[:, 0:3].sum(axis=1)
        # reorder = list(df_tss)[-2:-1] + list(df_tss)[:-2] + list(df_tss)[-1:]
        # df_tss = df_tss.loc[:, reorder]

        # Summing from the TSS to the right of the sample.
        df_tss = df.iloc[:, 62:]
        df_tss.loc[:, "ENSID"] = gene_list.loc[:, "Ensembl_ID"]
        df_tss.loc[:, "Sum"] = df_tss.iloc[:, :-1].sum(axis=1)
        reorder = list(df_tss)[-2:-1] + list(df_tss)[:-2] + list(df_tss)[-1:]
        df_tss = df_tss.loc[:, reorder]

        if cell_line == "K562":
            K562_files.append(df_tss.loc[:, "Sum"])
        elif cell_line == "GM12878":
            GM12878_files.append(df_tss.loc[:, "Sum"])

        plt.plot(range(len(df_tss)), df_tss.loc[: ,"Sum"])
        plt.xlabel("Gene")
        plt.ylabel("Sum of Bins")
        plt.title("Genes vs Sum of the TSS and the Bins to the Right")
        plt.savefig(f"/home/coder/REU/plots/dist_of_3tss_sum/{file_name}.sum_of_3tss_bins.png")
        plt.clf()

File: 100%|██████████| 8/8 [00:05<00:00,  1.57it/s]


<Figure size 640x480 with 0 Axes>

### Correlation

In [60]:
# Plotting the correlation values of each file and its ground truth value in RNA_CAGE.txt

# Creating a dataframe that contains the 3-bin-sum for each file, the ENSID, and the ground truth value.
gm12878 = pd.DataFrame(GM12878_files).T
gm12878.insert(0, "a", GM12878_gene_list["Ensembl_ID"])
gm12878.columns = range(len(gm12878.columns))
gm12878 = gm12878.merge(rna_cage, left_on=0, right_on="ENSID", how="left")
gm12878 = gm12878.loc[:, (list(gm12878)[0:5] + list(gm12878)[7:8])]
gm12878["GM12878_CAGE_128*3_sum"] = gm12878["GM12878_CAGE_128*3_sum"].astype(float).fillna(0)
gm12878.to_csv("/home/coder/REU/csvs/gm12878.csv", index=False)

for col in gm12878.columns[1:5]:
    file_name = combined_tab_files_name_sorted[int(col) - 1][ :combined_tab_files_name_sorted[int(col) - 1].find("_")]
    plt.scatter(gm12878[col], gm12878["GM12878_CAGE_128*3_sum"])

    plt.xscale("log")
    plt.yscale("log")

    plt.ylabel("Actual Value (Log)")
    plt.xlabel("Ground Truth (Log)")
    plt.title(f"Scatter Plot for Correlation of {file_name} vs RNA_CAGE", fontsize=8)
    plt.savefig(f"/home/coder/REU/plots/rnaseq_vs_rnacage/{file_name}.correlation_rna_cage_log.png")
    plt.clf()

k562 = pd.DataFrame(K562_files).T
k562.insert(0, "a", K562_gene_list["Ensembl_ID"])
k562.columns = range(len(k562.columns))
k562 = k562.merge(rna_cage, left_on=0, right_on="ENSID", how="left")
k562 = k562.loc[:, (list(k562)[0:5] + list(k562)[6:7])]
k562["K562_CAGE_128*3_sum"] = k562["K562_CAGE_128*3_sum"].astype(float).fillna(0)
k562.to_csv("/home/coder/REU/csvs/k562.csv", index=False)

for col in k562.columns[1:5]:
    file_name = combined_tab_files_name_sorted[int(col) - 1 + 4][ :combined_tab_files_name_sorted[int(col) - 1 + 4].find("_")]
    plt.scatter(k562[col], k562["K562_CAGE_128*3_sum"])

    plt.yscale("log")
    plt.xscale("log")

    plt.ylabel("Actual Value (Log)")
    plt.xlabel("Ground Truth (Log)")
    plt.title(f"Scatter Plot for Correlation of {file_name} vs RNA_CAGE", fontsize=8)
    plt.savefig(f"/home/coder/REU/plots/rnaseq_vs_rnacage/{file_name}.correlation_rna_cage_log.png")
    plt.clf()

<Figure size 640x480 with 0 Axes>

In [62]:
# Generating the dataframes for the correlation and p-value values (RNA Seq vs RNA Cage).
GM12878_corr_df = pd.DataFrame()
for col in gm12878.columns[1:5]:
    pearson_r_values = stats.pearsonr(np.log(gm12878[col] + 1), np.log(gm12878["GM12878_CAGE_128*3_sum"] + 1))
    # pearson_r_values = stats.pearsonr(gm12878[col], gm12878["GM12878_CAGE_128*3_sum"])
    GM12878_corr_df = pd.concat([GM12878_corr_df, pd.DataFrame(pearson_r_values).T])
GM12878_corr_df = GM12878_corr_df.rename(columns={0: "PearsonR", 1: "PValue"})
GM12878_corr_df["Sample"] = [file for file in combined_tab_files_names if file[0] == "G"]
GM12878_corr_df.to_csv("/home/coder/REU/csvs/correlation/rnaseq_vs_rnacage/GM12878.correlation_pvalue_log.csv")

# Pearson R: 0.14500713488288372
# PValue: 1.2354172154320445e-17

# Pearson R (Log): 0.13537539028281445
# PValue (Log): 4.208102991973101e-79
print(f"GM12878 Average PearsonR: {GM12878_corr_df.iloc[:, 0:2].mean().iloc[0]}\nAverage PValue: {GM12878_corr_df.iloc[:, 0:2].mean().iloc[1]}")

K562_corr_df = pd.DataFrame()
for col in k562.columns[1:5]:
    pearson_r_values = stats.pearsonr(np.log(k562[col] + 1), np.log(k562["K562_CAGE_128*3_sum"] + 1))
    K562_corr_df = pd.concat([K562_corr_df, pd.DataFrame(pearson_r_values).T])
K562_corr_df = K562_corr_df.rename(columns={0: "PearsonR", 1: "PValue"})
K562_corr_df["Sample"] = [file for file in combined_tab_files_names if file[0] == "K"]

K562_corr_df.to_csv("/home/coder/REU/csvs/correlation/rnaseq_vs_rnacage/K562.correlation_pvalue_log.csv")

# Pearson R: 0.24022424698280365
# PValue: 8.614336307937462e-100

# Pearson R (Log): 0.14187766207637342
# PValue (Log): 3.3351711137892027e-110 
print(f"K562 Average PearsonR: {K562_corr_df.iloc[:, 0:2].mean().iloc[0]}\nAverage PValue: {K562_corr_df.iloc[:, 0:2].mean().iloc[1]}")

GM12878 Average PearsonR: 0.13537539028281445
Average PValue: 4.208102991973101e-79
K562 Average PearsonR: 0.14187766207637342
Average PValue: 3.3351711137892027e-110


In [64]:
# Plotting RNA Cage vs Xpresso
rna_cage = pd.read_csv("/home/coder/data/RNA_CAGE.txt", sep="\t")
xpresso = pd.read_csv("/home/coder/data/GM12878_K562_18377_gene_expr_fromXpresso.csv")

epi_xpresso = rna_cage.merge(xpresso, left_on="ENSID", right_on="gene_id", how="inner")
epi_xpresso = epi_xpresso.loc[:, (list(epi_xpresso)[0:3] + list(epi_xpresso)[-7:-6] + list(epi_xpresso)[-5:-4])]
epi_xpresso.iloc[:, 1:5] = epi_xpresso.iloc[:, 1:5].astype(float).fillna(0)

cage_xpresso = pd.DataFrame()

plt.scatter(epi_xpresso["GM12878_CAGE_128*3_sum_y"], epi_xpresso["GM12878_CAGE_128*3_sum_x"])
plt.xlabel("Xpresso Values (Log)")

plt.yscale("log")
plt.xscale("log")

plt.ylabel("RNA Cage Values (Log)")
plt.title("GM12878 Xpresso vs RNA Cage Values (Log)")
plt.savefig("/home/coder/REU/plots/rnacage_vs_xpresso/GM12878.rna_cage_xpresso_inner_log.png")
plt.clf()

corr = stats.pearsonr(np.log(epi_xpresso["GM12878_CAGE_128*3_sum_y"] + 1), np.log(epi_xpresso["GM12878_CAGE_128*3_sum_x"] + 1))
cage_xpresso = pd.concat([cage_xpresso, pd.DataFrame(corr).T])

plt.scatter(epi_xpresso["K562_CAGE_128*3_sum_y"], epi_xpresso["K562_CAGE_128*3_sum_x"])
plt.xlabel("Xpresso Values (Log)")

plt.xscale("log")
plt.yscale("log")

plt.ylabel("RNA Cage Values (Log)")
plt.title("K562 Xpresso vs RNA Cage Values (Log)")
plt.savefig("/home/coder/REU/plots/rnacage_vs_xpresso/K562.rna_cage_xpresso_inner_log.png")
plt.clf()

corr = stats.pearsonr(np.log(epi_xpresso["K562_CAGE_128*3_sum_y"] + 1), np.log(epi_xpresso["K562_CAGE_128*3_sum_x"] + 1))
cage_xpresso = pd.concat([cage_xpresso, pd.DataFrame(corr).T])

cage_xpresso.to_csv("/home/coder/REU/csvs/correlation/rnacage_vs_xpresso/cage_xpresso.correlation_inner_log.csv", index=False)

<Figure size 640x480 with 0 Axes>

In [65]:
# Plotting RNA Seq vs Xpresso
gm12878 = pd.read_csv("/home/coder/REU/csvs/gm12878.csv")
k562 = pd.read_csv("/home/coder/REU/csvs/k562.csv")
xpresso = pd.read_csv("/home/coder/data/GM12878_K562_18377_gene_expr_fromXpresso.csv")

gm_xpresso = gm12878.merge(xpresso, left_on="0", right_on="gene_id", how="inner")
gm_xpresso = gm_xpresso.loc[:, (list(gm_xpresso)[0:14] + list(gm_xpresso)[-5:-4])]
gm_xpresso["GM12878_CAGE_128*3_sum_y"] = gm_xpresso["GM12878_CAGE_128*3_sum_y"].astype(float).fillna(0)

k5_xpresso = k562.merge(xpresso, left_on="0", right_on="gene_id", how="inner")
k5_xpresso = k5_xpresso.loc[:, (list(k5_xpresso)[0:14] + list(k5_xpresso)[-7:-6])]
k5_xpresso["K562_CAGE_128*3_sum_y"] = k5_xpresso["K562_CAGE_128*3_sum_y"].astype(float).fillna(0)

gm_xpresso_correlation = pd.DataFrame()
k5_xpresso_correlation = pd.DataFrame()

for col in gm_xpresso.columns[1:5]:
    file_name = combined_tab_files_name_sorted[int(col) - 1][ :combined_tab_files_name_sorted[int(col) - 1].find("_")]

    pearson_r_values = stats.pearsonr(np.log(gm_xpresso["GM12878_CAGE_128*3_sum_y"] + 1), np.log(gm_xpresso[col] + 1))
    gm_xpresso_correlation = pd.concat([gm_xpresso_correlation, pd.DataFrame(pearson_r_values).T])

    plt.scatter(gm_xpresso["GM12878_CAGE_128*3_sum_y"], gm_xpresso[col])
    plt.xlabel("Xpresso Values (Log)")

    plt.xscale("log")
    plt.yscale("log")

    plt.ylabel("RNA Seq Values (Log)")
    plt.title("GM12878 Xpresso vs RNA Seq Values")
    plt.savefig(f"/home/coder/REU/plots/rnaseq_vs_xpresso/{file_name}.rna_seq_xpresso_inner_log.png")
    plt.clf()

gm_xpresso_correlation = gm_xpresso_correlation.rename(columns={0: "PearsonR", 1: "PValue"})
gm_xpresso_correlation["Sample"] = [file for file in combined_tab_files_names if file[0] == "G"]
gm_xpresso_correlation.to_csv("/home/coder/REU/csvs/correlation/rnaseq_vs_xpresso/GM12878_Xpresso.correlation_pvalue_inner_log.csv", index=False)

for col in k5_xpresso.columns[1:5]:
    file_name = combined_tab_files_name_sorted[int(col) - 1 + 4][ :combined_tab_files_name_sorted[int(col) - 1 + 4].find("_")]

    pearson_r_values = stats.pearsonr(np.log(k5_xpresso["K562_CAGE_128*3_sum_y"] + 1), np.log(k5_xpresso[col] + 1))
    k5_xpresso_correlation = pd.concat([k5_xpresso_correlation, pd.DataFrame(pearson_r_values).T])

    plt.scatter(k5_xpresso["K562_CAGE_128*3_sum_y"], k5_xpresso[col])
    plt.xlabel("Xpresso Values (Log)")

    plt.xscale("log")
    plt.yscale("log")

    plt.ylabel("RNA Seq Values (Log)")
    plt.title("K562 Xpresso vs RNA Seq Values (Log)")
    plt.savefig(f"/home/coder/REU/plots/rnaseq_vs_xpresso/{file_name}.rna_seq_xpresso_inner_log.png")
    plt.clf()

k5_xpresso_correlation = k5_xpresso_correlation.rename(columns={0: "PearsonR", 1: "PValue"})
k5_xpresso_correlation["Sample"] = [file for file in combined_tab_files_names if file[0] == "K"]
k5_xpresso_correlation.to_csv("/home/coder/REU/csvs/correlation/rnaseq_vs_xpresso/K562_Xpresso.correlation_pvalue_inner_log.csv",index=False)

<Figure size 640x480 with 0 Axes>

## COVERAGE FILES

In [ ]:
# Generating the lineplot of the average signal value across the target interval from each coverage file.
enhancer_gene_K562_100kb = pd.read_csv("/home/coder/data/K562_enhancer_gene_links_100kb.hg38.tsv", sep="\t")
enhancer_gene_GM12878_100kb = pd.read_csv("/home/coder/data/GM12878_enhancer_gene_links_100kb.hg38.tsv", sep="\t")

gene_K562_tss = pd.read_csv("/home/coder/data/ABC-multiTSS_nominated/K562/Neighborhoods/GeneList.txt", sep="\t")[['name', 'Ensembl_ID', 'chr', 'tss', 'strand', 'H3K27ac.RPM.TSS1Kb', 'DHS.RPM.TSS1Kb']]
gene_GM12878_tss = pd.read_csv("/home/coder/data/ABC-multiTSS_nominated/GM12878/Neighborhoods/GeneList.txt", sep="\t")[['name', 'Ensembl_ID', 'chr', 'tss', 'strand', 'H3K27ac.RPM.TSS1Kb', 'DHS.RPM.TSS1Kb']]

for file in tqdm(coverage_files, desc="File"):
    cell_line = file[ :file.find(".")]
    file_name = file[ :file.find("E") + 11]
    enhancer_gene_100kb = None
    gene_tss = None

    if cell_line == "K562":
        enhancer_gene_100kb = enhancer_gene_K562_100kb
        gene_tss = gene_K562_tss
    elif cell_line == "GM12878":
        enhancer_gene_100kb = enhancer_gene_GM12878_100kb
        gene_tss = gene_GM12878_tss

    rna_df = pd.read_csv(f"/home/coder/data/RNASeq_bw/{file}", header=None, sep="\t")
    rna_df.columns = range(len(rna_df.columns))

    plus_rna_df = rna_df[rna_df[5] == "+"]
    minus_rna_df = rna_df[rna_df[5] == "-"]
    
    gene_tss["ENSID"] = gene_tss["Ensembl_ID"]
    enhancer_gene_100kb_includeNoEnhancerGene = enhancer_gene_100kb.merge(gene_tss, left_on="TargetGeneEnsembl_ID", right_on="Ensembl_ID", how="right", suffixes=["", "_gene"]).reset_index()
    gene_list = list(gene_tss["ENSID"])

    gene_enhancer_table = enhancer_gene_100kb_includeNoEnhancerGene
    rna_method = "encoding"

    rna_df_list = []
    plus_rna_df_list = []
    minus_rna_df_list = []

    for gene in tqdm(gene_list, desc="Gene"):
        gene_df = gene_enhancer_table[gene_enhancer_table["ENSID"] == gene]
        gene_enhancer_df = gene_df

        if rna_method is not None:
            if rna_method == "encoding" or rna_method == "one-hot":
                gene_rna_df = rna_df[rna_df[3] == gene]
                plus_gene = plus_rna_df[plus_rna_df[3] == gene]
                minus_gene = minus_rna_df[minus_rna_df[3] == gene]
            if rna_method == "embedding":
                gene_rna_df = rna_df[gene_list.index(gene)]
                plus_gene = plus_rna_df[gene_list.index(gene)]
                minus_gene = minus_rna_df[gene_list.index(gene)]

            # Grabs the DNA sequence.
            fasta_path = "/home/coder/data/hg38.fa"
            fasta_extractor = FastaStringExtractor(fasta_path)

            # Promoter will be the first row, then enhancers will be sorted by distance from promoter (increasing).
            gene_pe = gene_enhancer_df.sort_values(by="distance")
            row_0 = gene_pe.iloc[0]
            gene_ensid = row_0["TargetGeneEnsembl_ID"]
            gene_name = row_0["TargetGene"]
            gene_tss = row_0["TargetGeneTSS"]
            chrom = row_0["chr"]

            if row_0["TargetGeneTSS"] != row_0["TargetGeneTSS"]:
                gene_tss = row_0["tss"]
                gene_name = row_0["name_gene"]
                chrom = row_0["chr_gene"]

            # Grabs the DNA sequence of our TSS interval.
            max_seq_len = 2000
            target_interval = kipoiseq.Interval(chrom, int(gene_tss - max_seq_len / 2), int(gene_tss + max_seq_len / 2))
            promoter_seq = fasta_extractor.extract(target_interval)
            promoter_code = one_hot_encode(promoter_seq)

            # Grabs the RNA signals from each enhancer.
            if rna_method == "encoding" or rna_method == "one-hot":
                gene_len = row_0["end"] - row_0["start"]
                rna_signal = gene_rna_df[[9]]
                gene_rna_df = gene_rna_df[(gene_rna_df[7] >= target_interval.start) & (gene_rna_df[8] <= target_interval.end)]
                new_index = gene_rna_df[7].values - target_interval.start
                rna_signal = rna_signal.set_index(new_index).reindex(list(range(0, max_seq_len)), fill_value=0)
                gene_rna_df = np.array(rna_signal).flatten()

                plus_rna_signal = plus_gene[[9]]
                plus_gene = plus_gene[(plus_gene[7] >= target_interval.start) & (plus_gene[8] <= target_interval.end)]
                new_index = plus_gene[7].values - target_interval.start
                plus_rna_signal = plus_rna_signal.set_index(new_index).reindex(list(range(0, max_seq_len)), fill_value=0)
                plus_gene = np.array(plus_rna_signal).flatten()

                minus_rna_signal = minus_gene[[9]]
                minus_gene = minus_gene[(minus_gene[7] >= target_interval.start) & (minus_gene[8] <= target_interval.end)]
                new_index = minus_gene[7].values - target_interval.start
                minus_rna_signal = minus_rna_signal.set_index(new_index).reindex(list(range(0, max_seq_len)), fill_value=0)
                minus_gene = np.array(minus_rna_signal).flatten()
            if rna_method == "embedding":
                gene_rna_df = np.concatenate([gene_rna_df.reshape(1, 125), np.zeros([60, 125])])

                plus_gene = np.concatenate([plus_gene.reshape(1, 125), np.zeros([60, 125])])

                minus_gene = np.concatenate([minus_gene.reshape(1, 125), np.zeros([60, 125])])

            rna_df_list.append(gene_rna_df)
            plus_rna_df_list.append(plus_gene)
            minus_rna_df_list.append(minus_gene)

    df = pd.DataFrame(rna_df_list)
    plus_df = pd.DataFrame(plus_rna_df_list)
    minus_df = pd.DataFrame(minus_rna_df_list)

    df.to_csv("/home/coder/REU/RNA-Signal_csv/{file_name}.csv")
    plus_df.to_csv(f"/home/coder/REU/Plus-Minus-RNA_csv/GM12878.minus.ENCFF074SXQ.plus.csv")
    minus_df.to_csv(f"/home/coder/REU/Plus-Minus-RNA_csv/GM12878.minus.ENCFF074SXQ.minus.csv")

    # Plotting the average RNA signal for the entire file.
    plt.plot(df.columns, df.mean())
    plt.xticks(range(0, 2000, 100))
    plt.xlabel("Position in 2Kbp Promoter Window")
    plt.ylabel("Averaged RNA Signal")
    plt.title(f"Distribution of Averaged RNA Signals for {file_name}")
    plt.savefig(f"/home/coder/REU/plots/{file_name}.average_rna_signal.png")
    plt.clf()

    # Plotting just the plus dataframe.
    plt.plot(plus_df.columns, plus_df.mean())
    plt.xticks(range(0, 2000, 200))
    plt.xlabel("position in 2kbp promoter window")
    plt.ylabel("averaged rna signal")
    plt.title(f"distribution of averaged rna signals for gm12878.minus.encff074sxq.plus", fontsize=8)
    plt.savefig(f"/home/coder/reu/plots/{file_name}.plus.average_rna_signals.png")
    plt.clf()

    # Plotting just the minus dataframe.
    plt.plot(minus_df.columns, minus_df.mean())
    plt.xticks(range(0, 2000, 200))
    plt.xlabel("position in 2kbp promoter window")
    plt.ylabel("averaged rna signal")
    plt.title(f"distribution of averaged rna signals for gm12878.minus.encff074sxq.minus", fontsize=8)
    plt.savefig(f"/home/coder/reu/plots/{file_name}.minus.average_rna_signals.png")
    plt.clf()

    # Plotting both dataframes against each other.
    plt.plot(plus_df.columns, plus_df.mean())
    plt.plot(minus_df.columns, minus_df.mean())
    plt.xticks(range(0, 2000, 200))
    plt.xlabel("position in 2kbp promoter window")
    plt.ylabel("averaged rna signal")
    plt.title(f"distribution of averaged rna signals for gm12878.minus.encff074sxq plus and minus strands", fontsize=8)
    plt.savefig(f"/home/coder/reu/plots/{file_name}.plus_minus.average_rna_signals.png")
    plt.clf()